In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

from keras import models
from keras.preprocessing import sequence
import tensorflow as tf

import cv2

import os

C:\Users\never\Anaconda3\envs\tensorflow35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
file_path = "C:\\Users\\never\Workplace\\kaggle\\defector detect\\dataset\\train"
train_list = os.listdir(file_path)

In [5]:
img = cv2.imread(file_path +'\\'+ train_list[1],0)

In [28]:
total_set = dict()

for i in train_list[1:]:
    temp = cv2.imread(file_path + "\\" + i,0)
    total_set[i] = temp.copy()

AttributeError: 'NoneType' object has no attribute 'copy'

In [42]:
total_set[train_list[1]].shape

(256, 1600)